In [0]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
review = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Data_sets/movie_reviews.csv')


In [95]:
review.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [0]:
review['review'] = review['review'].apply(lambda x: x.lower())
review['sentiment'] = review['sentiment'].apply(lambda x:1 if x=='positive' else 0)


In [0]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

corpus = []
for i in range(0, len(review['review'])):
    review1 = re.sub('[^a-zA-Z]', ' ', str(review['review'][i]))
    review1 = review1.split()
    ## rootword running root is run
    ps = PorterStemmer()
    stopword_set = set(stopwords.words('english'))
    review1 = [ps.stem(word) for word in review1 if not word in stopword_set]
    review1 = ' '.join(review1)
    review['review'][i]=review1
    corpus.append(review1)


In [97]:
review.shape

(50000, 2)

In [0]:
from sklearn.model_selection  import train_test_split
X_train, X_test, y_train, y_test = train_test_split(review['review'], review['sentiment'],test_size=0.3, random_state=1)

In [99]:
X_train.str.len().max()

13704

In [0]:
from keras.preprocessing.text import Tokenizer
tokenizer= Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)

In [0]:
from keras.preprocessing.sequence import pad_sequences

X_train = pad_sequences(X_train, maxlen=13704)

In [102]:
print(X_train.shape)
print(y_train.shape)

(35000, 13704)
(35000,)


In [0]:
X_test = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(X_test, maxlen=13704)

In [77]:
print(X_test.shape)
print(y_test.shape)

(15000, 13704)
(15000,)


In [0]:
from keras.layers import Dense, Embedding,LSTM
from keras.models import Sequential

In [0]:
model = Sequential()
## Embdedding comes in 1st layer
model.add(Embedding(10000,128,mask_zero=True))
model.add(LSTM(128,dropout=0.4, recurrent_dropout=0.4,return_sequences=True))
#model.add(LSTM(64,dropout=0.5, recurrent_dropout=0.5,return_sequences=False))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#binary_crossentropy
#categorical_crossentropy

In [112]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, None, 128)         1280000   
_________________________________________________________________
lstm_12 (LSTM)               (None, None, 128)         131584    
_________________________________________________________________
dense_7 (Dense)              (None, None, 1)           129       
Total params: 1,411,713
Trainable params: 1,411,713
Non-trainable params: 0
_________________________________________________________________


In [113]:
X_train.shape

(35000, 13704)

In [114]:
y_train.shape

(35000,)

In [115]:
model.fit(X_train, y_train.reshape(35000,1,1), epochs=5, batch_size=12, verbose=1)

KeyboardInterrupt: ignored

In [117]:
pred_test = model.predict_classes(X_test)


KeyboardInterrupt: ignored